In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    print(list_of_files_in_genome_folder)
    if "Temp" in list_of_files_in_genome_folder:
        subprocess.run(f'rm -r "{genome_location}/{species}/Temp"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{genome_location}/{species}/Temp")
    return(genome_file)  

In [3]:
def create_query_folder(annotated_species_location,gene_name,species):
    import time


    list_of_files = os.listdir(annotated_species_location)
    if species in list_of_files:
        subprocess.run(f'rm -r "{annotated_species_location}/{species}"', shell = True, stderr = subprocess.DEVNULL)
    time.sleep(2.5)
    os.mkdir(f"{annotated_species_location}/{species}")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts/{gene_name}")
    
    

In [5]:
import os
import subprocess
from Bio import SeqIO

scaffold = ""
gene_name = "XM_042131467.1"
species = "Aricia_agestis"

genome_location = f"/mnt/f/Genomes_2023-12-26"
genome_file = get_genome_file(genome_location,species)
annotated_species_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/1.Annotated Species"
query_species = species
create_query_folder(annotated_species_location,gene_name,species)
list_of_files = os.listdir(f"{genome_location}/{species}")
print(list_of_files)
for file_names in list_of_files:
    if file_names.endswith("_genomic.gff"):
        gff3_file = file_names

with open(f"{genome_location}/{species}/{gff3_file}") as gff_file_open:
    gff_file_list = gff_file_open.readlines()
complement = ''
exon_count = 1
gene_found = 0
exon_dictionary = {}
for line in gff_file_list:
    if gene_name in line:
        if "CDS" in line.split("\t"):

            print(line)
            gene_found = 1
            if complement == '':
                complement = line.split("\t")[6]
            exon_start = line.split("\t")[3]
            exon_stop = line.split("\t")[4]
            scaffold = line.split("\t")[0]
            exon_dictionary[f"Exon_{exon_count}"] = [exon_start,exon_stop]
            exon_count += 1
        if "exon" in line.split("\t") and gene_found == 1:
            break
print(complement)
print(exon_dictionary)
if complement == "+":
    exon_counter = 1
elif complement == "-":
    exon_counter = len(exon_dictionary)
else:
    print("Error!!")
    assert False

# list_of_query_species = os.listdir(query_location)
# if query_species not in list_of_query_species:
#     os.mkdir(f"{annotated_species_location}/{query_species}")
# list_of_folders_here = os.listdir(f"{annotated_species_location}/{query_species}")
# if "Transcripts" not
#     os.mkdir(f"{annotated_species_location}/{query_species}/{gene_name}")

print_out = ''
print("exon counter", exon_counter)        
for i in range(1,len(exon_dictionary)+1):
    
    exon_start, exon_stop = exon_dictionary[f"Exon_{exon_counter}"]
#     print(exon_start,exon_stop)
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{exon_start}-{exon_stop} > "{genome_location}/{species}/Temp/temp_gene.fa"', shell = True)
    exon_sequence_file = SeqIO.parse(f"{genome_location}/{species}/Temp/temp_gene.fa", "fasta")
    sequence = ''
    
    for sequences in exon_sequence_file:
#         print(sequences.id)
        if complement == "-":
            sequence = sequences.seq.reverse_complement() 
            
#             print(sequences.seq.reverse_complement().translate())
        else:
            sequence = sequences.seq
#             print(sequences.seq)
#             print(sequences.seq.translate())
        print_out += str(len(sequence))+"\t"
        output = f">{query_species}_{gene_name}_Exon{exon_counter}\n{sequence}"
    with open(f"{annotated_species_location}/{query_species}/Transcripts/{gene_name}/Exon{exon_counter}.fa", 'w') as outfile:
        outfile.write(output)
    print(output)
    if complement == "-":
        exon_counter -= 1
    else:
        exon_counter += 1
                                            
#         if gene_location in line.split("\t"):
#           print(line)
#           break
print(print_out)

['GCF_905147365.1_ilAriAges1.1_genomic.fna', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.fai', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nhr', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nin', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nsq', 'GCF_905147365.1_ilAriAges1.1_genomic.gff', 'MD5SUMS', 'Temp']
['GCF_905147365.1_ilAriAges1.1_genomic.fna', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.fai', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nhr', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nin', 'GCF_905147365.1_ilAriAges1.1_genomic.fna.nsq', 'GCF_905147365.1_ilAriAges1.1_genomic.gff', 'MD5SUMS', 'Temp']
NC_056428.1	Gnomon	CDS	33839821	33839909	.	-	0	ID=cds-XP_041987401.1;Parent=rna-XM_042131467.1;Dbxref=GeneID:121739130,Genbank:XP_041987401.1;Name=XP_041987401.1;gbkey=CDS;gene=LOC121739130;product=period circadian protein;protein_id=XP_041987401.1

NC_056428.1	Gnomon	CDS	33826718	33826792	.	-	1	ID=cds-XP_041987401.1;Parent=rna-XM_042131467.1;Dbxref=GeneID:121739130,Genbank:XP_041987401.1;Name=XP_

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Aricia_agestis_XM_042131467.1_Exon23
CCGGCCCTAGTTAATGAGCAGCTCGGCCAGTTATACATCGACATGCAGCTCGAGGGTGTGGCTGCCAGGCTGACCCTGGAGGAGGGGATCACCAGCTCCGGGAGCTCGTCGGACGAGAACACCGCACATGCCTCCACG
>Aricia_agestis_XM_042131467.1_Exon22
GTATGTGTAACATCAGAACTCATTTATAAATATCAAATATTCAGCAAGACCATGGAAGAAGTGTTGTCAGGGGATAGAAAAAAGTTAGAAAAATTAGAACAG


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Aricia_agestis_XM_042131467.1_Exon21
TTCTGGCAAAGAAGACTTTTGATGGACGACGTGAGTAGCAATGCCAGTTGCGAACCGCTCCGGTCGCAGAACGTAACGAGAAGGAAAAAGGAACCCCCGTGGATGGAAAGA
>Aricia_agestis_XM_042131467.1_Exon20
AGCACCGACACAAACCAATTCGGCGAGAAGAAAGAAAACAGCGAGGGTACGATATTCAATACGGACGAGGACTTGAGTTTCTCTTCATTGTATTCCTCGTTCTTTAAGACTGAATCCAAAAGCGCTGAGGAGTCAGAACCGAAATCGGGAAATAAGGATGACGGCAAA


...

>Aricia_agestis_XM_042131467.1_Exon5
GACCTGGTATTCTGTATCAAGACGAAGGACTTTTTCCAGAGTACAGTATGTGACCCGCACGCTTCTTCTATAATTTGCACCATAGACTATCCCAATACACATTGTTCT
>Aricia_agestis_XM_042131467.1_Exon4
TTTACCTTAGTTCTCCACCGCCTACGGTTGGAGAAGGGGGCAATGAGTTAGATAACATGGACATATCTACGTCAGACAAAAGCACACGACAAAATAAA


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Aricia_agestis_XM_042131467.1_Exon3
CAAAAACAATAATGCACAACCTATTATAGAGAAAAGAGTTAATGATGAGATCAAAACTGCGGAAGTTAAACTAAAAAGCGCAGAAGTTGTTAATAATCATGATGCACAAACTATAGAATCTAAAGATACATCACCACCAACAACAGAAGATGTTAAAGATGGATCTGACAATG
>Aricia_agestis_XM_042131467.1_Exon2
TTCAAAATCTACGTCAGAGAGTAACTCGAGCGGCAGCAGTGGCTACCGAGGGAAACGCACTGATGCTGATTACAT


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Aricia_agestis_XM_042131467.1_Exon1
ATGGAGATTGTTGAAGATACAGAGAAACGTAGCGGAGTTGCTGACTCCGCTTATTCGTACAGTTGCAGCAACAGTAAATCTACTCAAAG
111	138	102	111	168	128	117	130	175	132	84	113	123	120	112	229	198	61	132	108	98	173	75	89	
